In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition
import matplotlib.pyplot as plt
import numpy as np
import re
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split

In [2]:
df=pd.read_csv('consumer_compliants.csv')

In [3]:
df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,04-03-2020,Vehicle loan or lease,Loan,Getting a loan or lease,Fraudulent loan,This auto loan was opened on XX/XX/2020 in XXX...,Company has responded to the consumer and the ...,TRUIST FINANCIAL CORPORATION,PA,None,None,Consent provided,Web,04-03-2020,Closed with explanation,Yes,NaN,3591341
1,03-12-2020,Debt collection,Payday loan debt,Attempts to collect debt not owed,Debt is not yours,In XXXX of 2019 I noticed a debt for {$620.00}...,None,CURO Intermediate Holdings,CO,806XX,None,Consent provided,Web,03-12-2020,Closed with explanation,Yes,NaN,3564184
2,02-06-2020,Vehicle loan or lease,Loan,Getting a loan or lease,Credit denial,"As stated from Capital One, XXXX XX/XX/XXXX an...",None,CAPITAL ONE FINANCIAL CORPORATION,OH,430XX,None,Consent provided,Web,02-06-2020,Closed with explanation,Yes,NaN,3521949
3,03-06-2020,Checking or savings account,Savings account,Managing an account,Banking errors,"Please see CFPB case XXXX. \n\nCapital One, in...",None,CAPITAL ONE FINANCIAL CORPORATION,CA,None,None,Consent provided,Web,03-06-2020,Closed with explanation,Yes,NaN,3556237
4,2/14/2020,Debt collection,Medical debt,Attempts to collect debt not owed,Debt is not yours,This debt was incurred due to medical malpract...,Company believes it acted appropriately as aut...,"Merchants and Professional Bureau, Inc.",OH,432XX,None,Consent provided,Web,2/14/2020,Closed with explanation,Yes,NaN,3531704


In [5]:
df['Product'].value_counts()

Debt collection                21772
Credit card or prepaid card    13193
Mortgage                        9799
Checking or savings account     7003
Student loan                    2950
Vehicle loan or lease           2736
Name: Product, dtype: int64

In [6]:
df['Company'].value_counts()

CITIBANK, N.A.                              3226
CAPITAL ONE FINANCIAL CORPORATION           2711
BANK OF AMERICA, NATIONAL ASSOCIATION       2580
JPMORGAN CHASE & CO.                        2409
WELLS FARGO & COMPANY                       2001
                                            ... 
Relin, Goldstein & Crane, LLP                  1
CMM of Louisiana, LLC                          1
Student Account Management Services, LLC       1
FLORIDA HOME TRUST MORTGAGE                    1
E. Margriet Langenberg, P.C.                   1
Name: Company, Length: 2197, dtype: int64

In [7]:
complaints_df=df[['Consumer complaint narrative','Product','Company']].rename(columns={'Consumer complaint narrative':'complaints'})

In [22]:
pd.set_option('display.max_colwidth', None)
complaints_df

,complaints,Product,Company
0,"This auto loan was opened on XX/XX/2020 in XXXX, NC with BB & T in my name. I have NEVER been to North Carolina and I have NEVER been a resident. I have filed a dispute twice through my credit bureaus but both times BB & T has claimed that this is an accurate loan. Which I wasn't aware of until today. I have tried to contact BB & T multiple times but I have never gotten through to a live person. I do n't drive and I have never owned a car before. I didn't have any knowledge of this account until I checked XXXXXXXX XXXX and noticed it. I've tried twice to dispute it. Additionally I never received any bills or information about this account. This is my last resort in trying to remove this fraudulent loan off of my account.",Vehicle loan or lease,TRUIST FINANCIAL CORPORATION
1,"In XXXX of 2019 I noticed a debt for {$620.00} on my credit which i believed was mine I thought speedy cash had bought one of my old debts and sold it to XXXX XXXX XXXX XXXX. I contacted XXXX XXXX XXXX XXXX and after several attempts of giving my full name, nothing came up in their system. I gave my social and the rep said the account popped up but DID NOT tell me that the account was under someone elses name and continued to let me make a payment. The payment was for {$120.00}. Confirmation number-XXXX. After realizing it was not my account, I called back to get my money back and inform them of the mistake. I was told i needed to mail them an FTC report and dispute letter to get my money back. I completed all of this and when i called again they said they transferred the account back to speedy cash for fraud review and I would need to contact them. After contacting them i was again told that i can not get my money back. The issue im having is this representative at XXXX XXXX played blind to obvious fraud and let an innocent person make a payment on someone elses debt and i want my money back.",Debt collection,CURO Intermediate Holdings
2,"As stated from Capital One, XXXX XX/XX/XXXX and XXXX 2018, My wife and I went to several car dealerships to request for a car loan to get a used car. However, according to their credit requirements unfortunately my credit score was insufficient for the car loan approval at that time. It seemed as though they pulled my credit report multiple times.",Vehicle loan or lease,CAPITAL ONE FINANCIAL CORPORATION
3,"Please see CFPB case XXXX. \n\nCapital One, in the letter they provided ( and attached to that case as their response ) said this : "" The funds were reversed and sent back to XXXX XXXX XXXX on XX/XX/XXXX ''. \n\nXXXX XXXX XXXX ( now XXXX XXXX ) has not received these funds. Staff at XXXX XXXX - and also staff at the account-holder 's business - have looked for return of my money ( {$650.00} ) and find nothing. \n\nCapital One needs to document - actually prove - they returned the funds, as stated in their letter. Capital One must provide electronic information, if the return was made that way, or document the paper check they sent back to XXXX XXXX. \n\nI've left 3 messages about this problem for the person who signed the letter ( XXXX ) from Capital One. I have received no call-backs. \n\nSummary : Capital One said they returned my money on XX/XX/XXXX : they did not. If they continue claim they did, then they need to prove that.",Checking or savings account,CAPITAL ONE FINANCIAL CORPORATION
4,"This debt was incurred due to medical malpractice ( XXXX XXXX XXXX, XXXX, TX ). I asked the doctor to turn over my claim to his malpractice insurance company. This has cost me thousands of dollars to XXXX XXXX XXXX. I am still trying to collect damages from this doctor. He never responded and turned over me to collections Merchants and Professional Collection Bureau , Inc. I sent them a letter describing exactly this issue and instead of not contacting me and verifying my debt they start reporting this debt to the credit reporting agencies. They never verified the debt, like I asked and they never

In [9]:
X_train, X_hold = train_test_split(complaints_df, test_size=0.6, random_state=111)

In [10]:
X_train['Product'].value_counts()

Debt collection                8720
Credit card or prepaid card    5297
Mortgage                       3809
Checking or savings account    2822
Student loan                   1236
Vehicle loan or lease          1097
Name: Product, dtype: int64

In [11]:
stemmer = PorterStemmer()

In [12]:
def tokenize(text):
   tokens = [word for word in nltk.word_tokenize(text) if (len(word) > 3 and len(word.strip('Xx/')) > 2) ] 
   #stems = [stemmer.stem(item) for item in tokens]
   return tokens

In [13]:
vectorizer_tf = TfidfVectorizer(tokenizer=tokenize, stop_words='english', max_df=0.75, min_df=50, max_features=10000, use_idf=False, norm=None)
tf_vectors = vectorizer_tf.fit_transform(X_train.complaints)

In [14]:
lda = decomposition.LatentDirichletAllocation(n_components=6, max_iter=3, learning_method='online', learning_offset=50, n_jobs=-1, random_state=111)
W1 = lda.fit_transform(tf_vectors)
H1 = lda.components_

In [15]:
num_words=15
vocab = np.array(vectorizer_tf.get_feature_names())
top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_words-1:-1]]
topic_words = ([top_words(t) for t in H1])
topics = [' '.join(t) for t in topic_words]

In [16]:
topics

['letter information complaint account request documents application provided credit received date document provide response documentation',
 'card credit account balance payment charges charge dispute bank late charged paid fees citi received',
 'loan payment mortgage payments home paid time insurance loans month company late months make escrow',
 'account bank chase money funds check checking america accounts deposit fraud xx/xx/2019 branch closed opened',
 'debt credit account report collection company information reporting letter agency sent original provide received reported',
 'told called said phone received number asked sent company time information account letter money calls']

In [17]:
colnames = ["Topic" + str(i) for i in range(lda.n_components)]
docnames = ["Doc" + str(i) for i in range(len(X_train.complaints))]
df_doc_topic = pd.DataFrame(np.round(W1, 2), columns=colnames, index=docnames)
significant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic['dominant_topic'] = significant_topic

In [18]:
df_doc_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,dominant_topic
Doc0,0.00,0.38,0.27,0.04,0.00,0.31,1
Doc1,0.26,0.00,0.00,0.73,0.00,0.00,3
Doc2,0.01,0.17,0.01,0.16,0.01,0.65,5
Doc3,0.10,0.00,0.00,0.00,0.37,0.52,5
Doc4,0.00,0.00,0.00,0.00,0.78,0.21,4
...,...,...,...,...,...,...,...
Doc22976,0.00,0.56,0.00,0.29,0.00,0.15,1
Doc22977,0.00,0.33,0.11,0.34,0.22,0.00,3
Doc22978,0.00,0.38,0.00,0.26,0.00,0.35,1
Doc22979,0.02,0.92,0.02,0.02,0.02,0.02,1


In [19]:
WHold = lda.transform(vectorizer_tf.transform(X_hold.complaints[:5]))

In [20]:
colnames = ["Topic" + str(i) for i in range(lda.n_components)]
docnames = ["Doc" + str(i) for i in range(len(X_hold.complaints[:5]))]
df_doc_topic = pd.DataFrame(np.round(WHold, 2), columns=colnames, index=docnames)
significant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic['dominant_topic'] = significant_topic

In [21]:
df_doc_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,dominant_topic
Doc0,0.03,0.16,0.00,0.81,0.00,0.00,3
Doc1,0.00,0.00,0.43,0.00,0.29,0.28,2
Doc2,0.01,0.70,0.01,0.14,0.01,0.15,1
Doc3,0.00,0.00,0.80,0.00,0.00,0.19,2
Doc4,0.00,0.76,0.14,0.00,0.00,0.10,1
